In [15]:
import pandas as pd
import os
import numpy as np
np.set_printoptions(precision=4)

import catboost
from catboost import *
from catboost import Pool
from catboost import CatBoostClassifier


import matplotlib.pyplot as plt
import geopandas as gpd

# other things: 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score, recall_score, f1_score, confusion_matrix, precision_score

!jupyter nbextension enable --py widgetsnbextension


Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
# in this notebook varya is trying to buld a "best-working" catboost classifier for the DF and FF catchments 
# then "deconstrust" that model for the shap values to see how different features affect the result of the model 
# PD? 



### 1. read the data, defince X and y, defince categorical features for y 

In [6]:
# read the data: 

path = '/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/ml_model/data_current/'
df = pd.read_csv(path + '09082022dfclim_permafrost.csv', sep = ',', index_col = 0)

df = df[df.target.isin([1,0])]
df = df.drop(['DN', 'Name', 'mean_daylymonsoon_tp_x', 'region_name'], axis = 1)


In [11]:
y = df.target
X = df.drop(columns=['target'])


# # what values are categorical: 
# # region, glacier, avgtemp_belowzero, cont_permafrost, any_permafrost, cont_permafrost_frac50
X['glacier'] = X['glacier'].astype('category') # n 21
X['region'] = X['region'].astype('category') # n 12
X['avgtemp_belowzero'] = X['avgtemp_belowzero'].astype('category') # n 34
X['cont_permafrost'] = X['cont_permafrost'].astype('category') # n 51
X['any_permafrost'] = X['any_permafrost'].astype('category') # n 52
X['cont_permafrost_frac50'] = X['cont_permafrost_frac50'].astype('category') # n 54

cat_features = list([12, 21, 34, 50, 51, 53])


### 2. classifier data prep: split to train and test, convert to pool data class 

In [12]:
# train test 
seed = 42
test_size = 0.3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


In [13]:
# pool

train_data = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_features,
                 )

test_data = Pool(data=X_test,
                 label=y_test,
                 cat_features=cat_features,
                 )

### 3. catboost classifier - build the model and identify the hyperparameters that are needed 

In [ ]:
# tune number of trees: 






In [ ]:
# import CatBoostClassifier
# logloss - default(?)
# CrossEntropy - for probabilities 

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    use_best_model= True,
    loss_function='CrossEntropy', 
    custom_loss=['AUC', 'Accuracy']
)


model.fit(
    train_data,                # pool thing - instead of X_train, y_train
    # cat_features=cat_features, # since the data is a pool - it already knows about it 
    eval_set=test_data,        # pool thing - instead of X_test, y_test
    verbose=False, 
    plot=True
)

print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())